In [1]:
import json
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from huggingface_hub import login


C:\Users\chuy-\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# -----------------------------
# 1. Cargar dataset
# -----------------------------
dataset = load_dataset("json", data_files="dataset.jsonl")

In [7]:
# -----------------------------
# 2. Cargar modelo base
# -----------------------------
model_name = "microsoft/Phi-3.5-mini-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="None",
)


ValueError: When passing device_map as a string, the value needs to be a device name (e.g. cpu, cuda:0) or 'auto', 'balanced', 'balanced_low_0', 'sequential' but found None.

In [4]:
# -----------------------------
# 3. Configurar LoRA
# -----------------------------
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)


In [5]:
# -----------------------------
# 4. Preprocesar el dataset
# -----------------------------
def format_instruction(example):
    prompt = f"Instrucción: {example['instruction']}\nRespuesta:"
    return tokenizer(prompt + example["response"], truncation=True)

tokenized = dataset.map(format_instruction)


In [6]:
# -----------------------------
# 5. Entrenamiento
# -----------------------------
training_args = TrainingArguments(
    output_dir="./lora-tutor",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    logging_steps=50,
    num_train_epochs=1,     # Pon 1 para no tardar horas
    fp16=False,             # IMPORTANTE
    bf16=False,
    save_steps=500,
    no_cuda=True,           # IMPORTANTÍSIMO
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

trainer.train()

C:\Users\chuy-\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\training_args.py:1636: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(
The model is already on multiple devices. Skipping the move to device specified in `args`.


RuntimeError: Function MmBackward0 returned an invalid gradient at index 1 - expected device meta but got cpu

In [ ]:
# -----------------------------
# 6. Guardar adaptadores LoRA
# -----------------------------
model.save_pretrained("./lora-tutor")
print("Entrenamiento completado. Adaptadores guardados.")